In [1]:
import numpy as np
import pandas as pd
from mne.stats import fdr_correction
import statsmodels.stats.multitest as multi
from scipy.stats import mannwhitneyu

In [2]:
df = pd.read_csv("CNV.Result.csv",sep=';', decimal=',')

In [3]:
df1 = df[df['Tumor'] == 1]
df2 = df[df['Tumor'] == 0]

In [4]:
#df1

In [5]:
#df2

In [6]:
PValue={}
Stat={}
alpha=0.05
df3=[]

In [7]:
for col in df:
    if col!="PatientID" and col !="Tumor":
        Stat[col],PValue[col] = mannwhitneyu(df1[col],df2[col])
        if PValue[col]<=alpha:
            df3.append({'Name':col,'PValue':PValue[col]})

In [8]:
df3=pd.DataFrame(df3)

In [9]:
df3

,Name,PValue
0,A1BG,7.676260e-13
1,A2M,1.832232e-02
2,A2MP1,3.097442e-02
3,NAT1,7.526055e-28
4,NAT2,7.783798e-28
5,AAMP,2.841886e-15
6,AANAT,1.169329e-53
7,AARS,3.371168e-02
8,ABAT,6.425952e-08
9,ABCA1,9.126119e-47


In [10]:
pval=df3['PValue']
pval=list(pval)
pval

[7.676259872507786e-13,
 0.01832231623290594,
 0.03097442240931975,
 7.52605483955858e-28,
 7.783797558647998e-28,
 2.8418861927397944e-15,
 1.1693291106657467e-53,
 0.03371167586884446,
 6.42595220342001e-08,
 9.12611898564769e-47,
 5.700688634353283e-33,
 2.0707977671643882e-05,
 4.176690433652693e-17,
 0.0018950618701955022,
 3.349432321571457e-36,
 6.0016416525588085e-112,
 4.474909184247112e-34,
 3.0764100833199035e-45,
 1.4581747679707422e-51,
 3.6672605825006995e-34,
 0.03475943694830241,
 5.166266301502674e-17,
 0.042775545501638444,
 3.0534405083575373e-07,
 7.309539685891835e-42,
 1.933073183236937e-43,
 2.0701845356602272e-26,
 4.4273435919630484e-07,
 4.038881027949963e-42,
 5.236935325428827e-32,
 5.410040886620541e-36,
 1.9534757399572628e-35,
 2.8686750116879007e-52,
 0.04036822843263206,
 3.821986630124487e-71,
 0.02544069577006724,
 1.4829456294838567e-99,
 6.260807315075563e-08,
 5.859113483980825e-75,
 1.0516481720927134e-49,
 1.6777488076040188e-56,
 5.5992557994661

In [11]:
PValue_holm = multi.multipletests(pval,alpha=alpha,method='holm')
Reject_holm = PValue_holm[0]
PValue_holm = PValue_holm[1]

In [12]:
for index,row in df3.iterrows():
 df3.at[index, 'reject_HOLM'] = Reject_holm.item(index)
 df3.at[index, 'PValue_HOLM'] = PValue_holm.item(index)

In [13]:
df3.reject_HOLM.value_counts()

True     14901
False     3433
Name: reject_HOLM, dtype: int64

In [14]:
PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_bh')
Reject_fdr = PValue_fdr[0]
PValue_fdr = PValue_fdr[1]


In [15]:
for index,row in df3.iterrows():
 df3.at[index, 'reject_fdr_bh'] = Reject_fdr.item(index)
 df3.at[index, 'PValue_fdr_bh'] = PValue_fdr.item(index)

In [16]:
PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_by')
Reject_fdr = PValue_fdr[0]
PValue_fdr = PValue_fdr[1]
for index,row in df3.iterrows():
 df3.at[index, 'reject_fdr_by'] = Reject_fdr.item(index)
 df3.at[index, 'PValue_fdr_by'] = PValue_fdr.item(index)


In [17]:
PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_tsbh')
Reject_fdr = PValue_fdr[0]
PValue_fdr = PValue_fdr[1]
for index,row in df3.iterrows():
 df3.at[index, 'reject_fdr_tsbh'] = Reject_fdr.item(index)
 df3.at[index, 'PValue_fdr_tsbh'] = PValue_fdr.item(index)


In [18]:
PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_tsbky')
Reject_fdr = PValue_fdr[0]
PValue_fdr = PValue_fdr[1]
for index,row in df3.iterrows():
 df3.at[index, 'reject_fdr_tskby'] = Reject_fdr.item(index)
 df3.at[index, 'PValue_fdr_tskby'] = PValue_fdr.item(index)


In [19]:
df3

,Name,PValue,reject_HOLM,PValue_HOLM,reject_fdr_bh,PValue_fdr_bh,reject_fdr_by,PValue_fdr_by,reject_fdr_tsbh,PValue_fdr_tsbh,reject_fdr_tskby,PValue_fdr_tskby
0,A1BG,7.676260e-13,True,5.013365e-09,True,1.192278e-12,True,1.239225e-11,True,1.192278e-12,True,4.779779e-16
1,A2M,1.832232e-02,False,1.000000e+00,True,1.922957e-02,False,1.998674e-01,True,1.922957e-02,True,7.709029e-06
2,A2MP1,3.097442e-02,False,1.000000e+00,True,3.174493e-02,False,3.299490e-01,True,3.174493e-02,True,1.272637e-05
3,NAT1,7.526055e-28,True,7.465846e-24,True,1.639723e-27,True,1.704288e-26,True,1.639723e-27,True,6.573559e-31
4,NAT2,7.783798e-28,True,7.720749e-24,True,1.695677e-27,True,1.762445e-26,True,1.695677e-27,True,6.797875e-31
5,AAMP,2.841886e-15,True,2.101291e-11,True,4.756541e-15,True,4.943832e-14,True,4.756541e-15,True,1.906871e-18
6,AANAT,1.169329e-53,True,1.814916e-49,True,7.599603e-53,True,7.898841e-52,True,7.599603e-53,True,3.046639e-56
7,AARS,3.371168e-02,False,1.000000e+00,True,3.440986e-02,False,3.576476e-01,True,3.440986e-02,True,1.379472e-05
8,ABAT,6.425952e-08,True,3.067107e-04,True,8.685742e-08,True,9.027748e-07,True,8.685742e-08,True,3.482066e-11
9,ABCA1,9.126119e-47,True,1.306221e-42,True,4.160076e-46,True,4.323881e-45,True,4.160076e-46,True,1.667752e-49


In [20]:
df3.to_csv(r'C:\Users\Macie\Praca_Dyplomowa\PValue.CNV.csv',sep=';', index=False)
#df3 = pd.read_csv("PValue.CNV.csv",sep=';', decimal=',')